<a href="https://colab.research.google.com/github/ahmedmabrouk24/Computer_vision/blob/main/verify_signature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [133]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from sklearn.linear_model import LogisticRegression
from skimage.feature import hog
from sklearn.svm import SVC
from keras.saving.save import load_model
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from sklearn.metrics import accuracy_score
import tensorflow as tf
import cv2
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import pickle
from PIL import Image
import shutil
import json


In [10]:
TRAIN_DIR_A = '/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Train'
TRAIN_DIR_B = '/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Train'
TRAIN_DIR_C = '/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Train'
TRAIN_DIR_D = '/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Train'
TRAIN_DIR_E = '/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Train'
TRAIN_DIR=[]
TRAIN_DIR.append(TRAIN_DIR_A)
TRAIN_DIR.append(TRAIN_DIR_B)
TRAIN_DIR.append(TRAIN_DIR_C)
TRAIN_DIR.append(TRAIN_DIR_D)
TRAIN_DIR.append(TRAIN_DIR_E)


Test_DIR_A = '/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Test'
Test_DIR_B = '/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Test'
Test_DIR_C = '/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Test'
Test_DIR_D = '/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Test'
Test_DIR_E = '/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Test'
Test_DIR=[]
Test_DIR.append(Test_DIR_A)
Test_DIR.append(Test_DIR_B)
Test_DIR.append(Test_DIR_C)
Test_DIR.append(Test_DIR_D)
Test_DIR.append(Test_DIR_E)

PersonA_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Train/personA_SigVerificationTrainLabels.csv'
PersonA_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Test/personA_SigVerificationTestLabels.csv'

PersonB_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Train/personB_SigVerificationTrainLabels.csv'
PersonB_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Test/personB_SigVerificationTestLabels.csv'

PersonC_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Train/personC_SigVerificationTrainLabels.csv'
PersonC_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Test/personC_SigVerificationTestLabels.csv'

PersonD_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Train/personD_SigVerificationTrainLabels.csv'
PersonD_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Test/personD_SigVerificationTestLabels.csv'

PersonE_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Train/personE_SigVerificationTrainLabels.csv'
PersonE_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Test/personE_SigVerificationTestLabels.csv'


IMG_SIZE = 200
num_classes = 5
MODEL_NAME = 'signature_model1'


In [11]:
def create_label(image_name,status):
    if image_name[6]=='A' and status=='train':
      df = pd.read_csv(PersonA_train_label)
    elif image_name[6]=='A' and status=='test':
      df = pd.read_csv(PersonA_test_label) 

    elif image_name[6]=='B' and status=='train':
       df = pd.read_csv(PersonB_train_label)
    elif image_name[6]=='B' and status=='test':
       df = pd.read_csv(PersonB_test_label)

    elif image_name[6]=='C' and status=='train':
       df = pd.read_csv(PersonC_train_label)
    elif image_name[6]=='C' and status=='test':
       df = pd.read_csv(PersonC_test_label)

    elif image_name[6]=='D' and status=='train':
       df = pd.read_csv(PersonD_train_label)
    elif image_name[6]=='D' and status=='test':
       df = pd.read_csv(PersonD_test_label)

    elif image_name[6]=='E' and status=='train':
       df = pd.read_csv(PersonE_train_label)
    elif image_name[6]=='E' and status=='test':
       df = pd.read_csv(PersonE_test_label)
    x=False
    for ind in df.index :
      if image_name==df['image_name'][ind]:
        x=True
        if df['label'][ind]=='real':
          return 1
        else:
          return 0
    if x==False:
      print(image_name)


In [12]:
x_train_imageA=[]
x_train_imageB=[]
x_train_imageC=[]
x_train_imageD=[]
x_train_imageE=[]

y_train_imageA=[]
y_train_imageB=[]
y_train_imageC=[]
y_train_imageD=[]
y_train_imageE=[]


x_test_imageA=[]
x_test_imageB=[]
x_test_imageC=[]
x_test_imageD=[]
x_test_imageE=[]

y_test_imageA=[]
y_test_imageB=[]
y_test_imageC=[]
y_test_imageD=[]
y_test_imageE=[]


In [20]:
def create_train_data():
    

    for path_TRAIN_DIR in tqdm(TRAIN_DIR):
       for img in os.listdir(path_TRAIN_DIR):
        if 'csv' in img:
          continue
        path = os.path.join(path_TRAIN_DIR, img)
        img_data = cv2.imread(path,0) 
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))

        label = create_label(img,'train')
        feature_discriptor, hog_img = hog(img_data, orientations = 9, pixels_per_cell = (8, 8),cells_per_block = (2, 2), visualize = True, multichannel = False)
        
        if img[6]=='A':
           x_train_imageA.append(feature_discriptor)
           y_train_imageA.append(label)
        elif img[6]=='B':
          x_train_imageB.append(feature_discriptor)
          y_train_imageB.append(label)   
        elif img[6]=='C':
           x_train_imageC.append(feature_discriptor)
           y_train_imageC.append(label)
        elif img[6]=='D':
           x_train_imageD.append(feature_discriptor)
           y_train_imageD.append(label)
        elif img[6]=='E':
           x_train_imageE.append(feature_discriptor)
           y_train_imageE.append(label)

    


In [18]:
column1=[]
def create_test_data():
    test_data = []

    for path_Test_DIR in tqdm(Test_DIR):
       for img in os.listdir(path_Test_DIR):
        if 'csv' in img:
          continue
        column1.append(img)
        path = os.path.join(path_Test_DIR, img)

        img_data = cv2.imread(path,0) 
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        feature_discriptor, hog_img = hog(img_data, orientations = 9, pixels_per_cell = (8, 8),cells_per_block = (2, 2), visualize = True, multichannel = False)
        label = create_label(img,'test')
        
        if img[6]=='A':
           x_test_imageA.append(feature_discriptor)
           y_test_imageA.append(label)
        elif img[6]=='B':
          x_test_imageB.append(feature_discriptor)
          y_test_imageB.append(label)   
        elif img[6]=='C':
           x_test_imageC.append(feature_discriptor)
           y_test_imageC.append(label)
        elif img[6]=='D':
           x_test_imageD.append(feature_discriptor)
           y_test_imageD.append(label)
        elif img[6]=='E':
           x_test_imageE.append(feature_discriptor)
           y_test_imageE.append(label)

    

In [21]:
create_train_data()
create_test_data()


100%|██████████| 5/5 [00:15<00:00,  3.15s/it]


In [130]:
DT = DecisionTreeClassifier()

DT.fit(x_train_imageA,y_train_imageA)
y_prid=DT.predict(x_test_imageA)
print(accuracy_score(y_test_imageA,y_prid)*100)

87.5


In [134]:
pickle.dump(DT, open('/content/drive/MyDrive/modelA', 'wb'))

In [131]:
MLB = MLPClassifier()
MLB.fit(x_train_imageB,y_train_imageB)
y_prid=MLB.predict(x_test_imageB)
print(accuracy_score(y_test_imageB,y_prid)*100)

87.5


In [135]:
pickle.dump(MLB, open('/content/drive/MyDrive/modelB', 'wb'))

In [139]:
KNC = KNeighborsClassifier()

KNC.fit(x_train_imageC,y_train_imageC)
y_prid=KNC.predict(x_test_imageC)
print(accuracy_score(y_test_imageC,y_prid)*100)

100.0


In [140]:
pickle.dump(KNC, open('/content/drive/MyDrive/modelC', 'wb'))

In [141]:
KNC2 = KNeighborsClassifier()
KNC2.fit(x_train_imageD,y_train_imageD)
y_prid=KNC2.predict(x_test_imageD)
print(accuracy_score(y_test_imageD,y_prid)*100)

87.5


In [142]:
pickle.dump(KNC2, open('/content/drive/MyDrive/modelD', 'wb'))

In [143]:
KNC3 = KNeighborsClassifier()

KNC3.fit(x_train_imageE,y_train_imageE)
y_prid=KNC3.predict(x_test_imageE)
print(accuracy_score(y_test_imageE,y_prid)*100)

100.0


In [144]:
pickle.dump(KNC3, open('/content/drive/MyDrive/modelE', 'wb'))